In [7]:
# Cell 1: Imports
import pandas as pd
from pathlib import Path
from deepfake_eval.loaders import ASVspoof2019LALoader
from deepfake_eval.path_utils import data_path

In [8]:
# Cell 2: Configuration, Load Data, and Calculate EER
MODEL_NAME = "AASIST"
DATASET_NAME = "ASVspoof2019 LA"

# --- IMPORTANT: Verify and update this path to your actual score file ---
SCORE_FILE_PATH = data_path("scores", "aasist", "aasist_asvspoof2019LAeval_scores.pkl")
# TRIAL_FILE_PATH = None # Let the loader use its default trial file path

CALIBRATED_THRESHOLD = None
df_scores = pd.DataFrame()

if not Path(SCORE_FILE_PATH).is_file():
    print(f"ERROR: Score file not found at '{SCORE_FILE_PATH}'")
else:
    try:
        loader = ASVspoof2019LALoader.from_pkl_dict(
            score_pkl=str(SCORE_FILE_PATH),
            model_name=MODEL_NAME,
            # trial_file=TRIAL_FILE_PATH # Specify if not using default
        )
        df_scores = loader.to_dataframe()

        if not df_scores.empty:
            print(f"Calculating EER for {MODEL_NAME} on {DATASET_NAME}...\n")
            
            eer, eer_threshold = loader.eer()
            CALIBRATED_THRESHOLD = eer_threshold

            num_bonafide = (df_scores['df_class'] == 'bonafide').sum()
            num_spoof = (df_scores['df_class'] == 'spoof').sum()

            print(f"--- Calibration Results ---")
            print(f"Model: {MODEL_NAME}")
            print(f"Dataset: {DATASET_NAME}")
            print(f"EER: {eer:.4f} (i.e., {eer*100:.2f}%)")
            print(f"Threshold for EER: {CALIBRATED_THRESHOLD:.4f}")
            print(f"Number of bona fide samples: {num_bonafide}")
            print(f"Number of spoof samples: {num_spoof}")
            print(f"---------------------------")
            print(f"\nCalibrated threshold for {MODEL_NAME} on {DATASET_NAME} is: {CALIBRATED_THRESHOLD:.4f}")
        else:
            print("Data loaded, but DataFrame is empty. Cannot calculate EER.")

    except FileNotFoundError as e:
        print(f"ERROR: Could not load scores. File not found: {e.filename}")
    except KeyError as e:
        print(f"ERROR: Could not load scores due to a key error: {e}")
        print("This might be due to missing utterances or incorrect score file format.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


Building records 2019 LA: 100%|██████████| 71237/71237 [00:00<00:00, 116678.24it/s]

Calculating EER for AASIST on ASVspoof2019 LA...

--- Calibration Results ---
Model: AASIST
Dataset: ASVspoof2019 LA
EER: 0.0083 (i.e., 0.83%)
Threshold for EER: 1.4938
Number of bona fide samples: 7355
Number of spoof samples: 63882
---------------------------

Calibrated threshold for AASIST on ASVspoof2019 LA is: 1.4938
